In [2]:
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.signal import windows,freqz
from scipy import signal, fft

In [1]:
#GaussianNoise
class Noise():
    def __init__(self, length, generatingNoise = None):
        #generatingNoise
        self.length = length
        if generatingNoise == 'Lomax':
            self.generatingNoise = (np.random.pareto(5,2*length)-1/4)*math.sqrt(27/5)
        elif generatingNoise == 'Gaussian':
            self.generatingNoise = np.random.normal(0,1,2*length)
        else:
            self.generatingNoise = generatingNoise

        
    def linearProcess(self, coeffs = [2,1,4,3]):
        #By default, coefficients are a_0=2,a_1=1,a_2=3,a_3=4
        #X_t = sum a_i e_(t-i)
        linearNoise = []
        for i in range(length):
            rv = 0
            j = 0
            for coeff in coeffs:
                rv += self.generatingNoise[i+j]*coeff
                j += 1
            linearNoise.append(rv)
        return np.array(linearNoise)
    
    def volterraSeries(self, terms = [[3,[1,2,4]], [7,[1,2,5]], [2,[5,7,8,9]], [3,[1,2,3,4,5]], [2,[3]]]):
        #[[Coefs,terms]] tells us the specific function (coeff) applied to the product of terms. By default is above
        volterraSeries = []
        for i in range(length):
            rv = 0
            for term in terms:
                coeff = term[0]
                gens = term[1]
                base = 1
                for gen in gens:
                    base *= self.generatingNoise[i+gen]
                rv += coeff*base
            volterraSeries.append(rv)
        return np.array(volterraSeries)
    
    def AR(self, coeffs = [2.7607, -3.8106, 2.6535, -0.9238]):
        #Simulating AR(4) noise, where the first few terms are fixed. 
        #We assume that after 1000 iterations the terms approach the real AR noise.
        AR = [self.generatingNoise[-1],self.generatingNoise[-2],self.generatingNoise[-3],self.generatingNoise[-4]]
        for i in range(length+math.floor(length/2)):
            Xi = coeffs[0]*AR[-1]+coeffs[1]*AR[-2]+coeffs[2]*AR[-3]+coeffs[3]*AR[-4]+self.generatingNoise[i]
            AR.append(Xi)
#         print(max(AR))
        return np.array(AR[math.floor(length/2)+4:])
    
    def general(self):
        #Trying something wild
        general = [self.generatingNoise[-1],self.generatingNoise[-2],self.generatingNoise[-3],self.generatingNoise[-4]]
        for i in range(length+math.floor(length/2)):
            Xi = general[-1]*general[-2]*self.generatingNoise[i]/10 + self.generatingNoise[i+1] + 5*np.e**(-general[-1]**2)
            general.append(Xi)
        return np.array(general[math.floor(self.length/2)+4:])
    
    def t_dist(self): 
        import scipy 
        output = scipy.stats.t.rvs(4, size=self.length)
        return np.array(output)

In [1]:
def get_MT_F_Statistic(signalSample, gridSize):
    zeros = np.zeros(int(length/2)+1)
    dftTapers = []
    for i in range(len(wins)):
        dftTapers.append(sum(wins[i]))
    taperPower = sum(x**2 for x in dftTapers)
    dftDataByWindows = []
    for i in range(len(wins)):
        dftDataByWindows.append(fft.fft(wins[i]*signalSample, n = gridSize))
    statistic = []
    sdfMod = []
    sampFreqs = []
    for i in range(len(dftDataByWindows[0])):
        tempSum = 0
        tempSum2 = 0
        for j in range(len(wins)):
            tempSum += dftTapers[j]*dftDataByWindows[j][i]/taperPower
        for j in range(len(wins)):
            tempSum2 += abs(dftDataByWindows[j][i]-dftTapers[j]*tempSum)**2
        statistic.append((len(wins)-1)*abs(tempSum)**2*taperPower/tempSum2)
        sdfMod.append(tempSum2/(len(wins)-1))
        sampFreqs.append(i/gridSize)
    statistic = statistic[:math.floor(gridSize/2)+1]
    return sampFreqs[:math.floor(gridSize/2)+1], sdfMod[:math.floor(gridSize/2)+1], statistic

In [3]:
def padded_fft(x, n):
    axis = 0
    n_original = n
    n_power_of_2 = 2 ** int(math.ceil(math.log(n_original, 2)))
    n_pad = n_power_of_2 - x.shape[1:][0]
    z = np.zeros( (n_pad,) + x.shape[1:] )
    padded = np.concatenate((x, z), axis=axis)
    return sp.fftpack.fft(padded, axis=axis)

def get_waveShapeFStat(signalSample, gridSize, minFreq, maxFreq, sigma):
    #For the case where number of multiples is known
    dftTapers = np.array([sum(wins[i]) for i in range(len(wins))])
    taperPower = sum(x**2 for x in dftTapers)
    x = np.array(wins[0]*signalSample).resize(gridSize)
    dftDataByWindows = np.array([sp.fftpack.fft(wins[i]*signalSample, gridSize) for i in range(len(wins))])
    
    statistic = np.zeros(len(dftDataByWindows[0]))
    sampFreqs = np.array([(i/gridSize) for i in range(len(dftDataByWindows[0]))])

    tempSum = np.dot(dftTapers,dftDataByWindows)/taperPower
    tempSum2 = dftDataByWindows-np.reshape(np.kron(dftTapers,tempSum), (len(dftTapers),len(dftDataByWindows[0])))
    tempSum2 = np.sum(np.abs(tempSum2)**2,axis=0)
    statistic = (len(wins)-1)*taperPower*np.square(np.abs(tempSum))/tempSum2   
    
    statistic2 = [None]*len(statistic)
    statistic2[0] = 0
    indices = [[i*j for i in range(1,int(len(statistic)/(2*j)))] for j in range(1,sigma+1)]
    statistic2[1:] = sum(np.pad(statistic[indices[j]], (0, len(statistic)-1-len(indices[j]))) for j in range(sigma))
    
    lowerBound = np.searchsorted(sampFreqs, minFreq)
    upperBound = np.searchsorted(sampFreqs, maxFreq)
    
    sampFreqs = sampFreqs[lowerBound: upperBound]
    statistic2 = statistic2[lowerBound: upperBound]
    max_fstat = max(statistic2)
    indexHat = np.argmax(statistic2)
    argmax_freq = sampFreqs[indexHat]
    
    return max_fstat, argmax_freq

def get_waveShapeFStat_unknownL(signalSample, gridSize, minFreq, maxFreq, sigmaMax, penalty):
    #For the case where number of multiples is unknown
    dftTapers = np.array([sum(wins[i]) for i in range(len(wins))])
    taperPower = sum(x**2 for x in dftTapers)
    x = np.array(wins[0]*signalSample).resize(gridSize)
    dftDataByWindows = np.array([sp.fftpack.fft(wins[i]*signalSample, gridSize) for i in range(len(wins))])
    
    statistic = np.zeros(len(dftDataByWindows[0]))
    sampFreqs = np.array([(i/gridSize) for i in range(len(dftDataByWindows[0]))])

    tempSum = np.dot(dftTapers,dftDataByWindows)/taperPower
    tempSum2 = dftDataByWindows-np.reshape(np.kron(dftTapers,tempSum), (len(dftTapers),len(dftDataByWindows[0])))
    tempSum2 = np.sum(np.abs(tempSum2)**2,axis=0)
    statistic = (len(wins)-1)*taperPower*np.square(np.abs(tempSum))/tempSum2   
    
    lowerBound = np.searchsorted(sampFreqs, minFreq)
    upperBound = np.searchsorted(sampFreqs, maxFreq)
    sampFreqs = sampFreqs[lowerBound: upperBound]
    
    max_fstat = -float("inf")
    argmax_freq = 0
    argmax_mult = 0
    
    for sigma in range(1,sigmaMax+1):
        statistic2 = [None]*len(statistic)
        statistic2[0] = 0
        indices = [[i*j for i in range(1,int(len(statistic)/(2*j)))] for j in range(1,sigma+1)]
        statistic2[1:] = sum(np.pad(statistic[indices[j]], (0, len(statistic)-1-len(indices[j]))) for j in range(sigma))
        statistics2 = statistic2[lowerBound: upperBound]

        indexHat = np.argmax(statistics2)
        fstat = max(statistics2)-sigma*penalty
        freq = sampFreqs[indexHat]
        if fstat > max_fstat: 
            max_fstat = fstat
            argmax_freq = freq
            argmax_mult = sigma

    return max_fstat, argmax_freq, argmax_mult

def remove_waveShape(signalSample, wins, thetaHat, LHat):
    length = len(signalSample)
    A_estimates = [0]
    
    taperCoeff = []
    for l in range(len(wins)):
        taperCoeff.append(sum(wins[l]))
    taperPower = sum(x**2 for x in taperCoeff)
    taperCoeff = np.array(taperCoeff)/taperPower
    
    for sigma in range(1,LHat+1):
        A_sigma_hat = 0
        for l in range(len(wins)):
            A_sigma_hat += taperCoeff[l]*sum(wins[l][i]*signalSample[i]*(math.cos(2*math.pi*thetaHat*i)
                                              -1j*math.sin(2*math.pi*thetaHat*i)) for i in range(length))
        A_estimates.append(A_sigma_hat)
        
    for sigma in range(1,LHat+1):
        signalSample -= abs(2*A_estimates[sigma])*np.cos(2*np.pi*np.arange(length)*thetaHat + np.angle(A_estimates[sigma]))
        
    return signalSample